# ===========================================================
# How to add new bundles into pyAFQ (Optic Radiations Example)
# ===========================================================

pyAFQ is designed to be customizable and extensible. This example shows how you
can customize it to define a new bundle based on a definition of waypoint and
endpoint ROIs of your design.

In this case, we add the optic radiations, based on work by Caffara et al.
The optic radiations (OR) are the primary projection of visual information
from the lateral geniculate nucleus of the thalamus to the primary visual
cortex. Studying the optic radiations with dMRI provides a linkage between white
matter tissue properties, visual perception and behavior, and physiological
responses of the visual cortex to visual stimulation.


In [ ]:
import os
import os.path as op

home = op.expanduser("~")
pwd = op.dirname(os.getcwd())

os.environ["TEMPLATEFLOW_HOME"] = op.join(home, "data_", "tractometry")
os.environ["DIPY_HOME"] = op.join(home, "data_", "tractometry")
afq_home = op.join(home, "data_", "tractometry")
os.makedirs(afq_home, exist_ok=True)
os.environ["AFQ_HOME"] = afq_home

In [ ]:
import os.path as op
import plotly
import numpy as np
from IPython.display import Image

from AFQ.api.group import GroupAFQ
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
from AFQ.definitions.image import ImageFile, RoiImage
import AFQ.utils.streamlines as aus
np.random.seed(1234)

## Get dMRI data

We will analyze one subject from the Healthy Brain Network Processed Open
Diffusion Derivatives dataset (HBN-POD2). We'll use a fetcher to
get preprocessed dMRI data for one of the >2,000 subjects in that study.


In [ ]:
study_dir = afd.fetch_hbn_preproc(["NDARAA948VFH"])[1]

## Define custom `BundleDict` object

The `BundleDict` object holds information about "include" and "exclude" ROIs,
as well as endpoint ROIs, and whether the bundle crosses the midline.


In [ ]:
or_rois = afd.read_or_templates()
bundles = abd.OR_bd()

## Define GroupAFQ object

For tractography, we use CSD-based probabilistic tractography seeding
extensively (`n_seeds=4` means 81 seeds per voxel!), but only within the ROIs.


In [ ]:
brain_mask_definition = ImageFile(
    suffix="mask",
    filters={'desc': 'brain',
             'space': 'T1w',
             'scope': 'qsiprep'})

my_afq = GroupAFQ(
    bids_path=study_dir,
    preproc_pipeline="qsiprep",
    participant_labels=["NDARAA948VFH"],
    output_dir=op.join(study_dir, "derivatives", "afq_or"),
    brain_mask_definition=brain_mask_definition,
    tracking_params={"n_seeds": 4,
                     "directions": "prob",
                     "odf_model": "CSD",
                     "seed_mask": RoiImage()},
    bundle_info=bundles)

my_afq.export_all()

## Visualize a montage

One way to examine the output of the pyAFQ pipeline is by creating a montage
of images of a particular bundle across a group of participants.


In [ ]:
my_afq.combine_bundle("Left Optic Radiation")
montage = my_afq.group_montage(
    "Left Optic Radiation",
    (1, 1), "Axial", "left")
Image(filename=montage[0])

## Interactive bundle visualization

Another way to examine the outputs is to export the individual bundle
figures, which show the streamlines, as well as the ROIs used to define the
bundle.


In [ ]:
bundle_html = my_afq.export("indiv_bundles_figures")
plotly.io.show(bundle_html["NDARAA948VFH"]["Left Optic Radiation"])

## References

- Caffarra S, Joo SJ, Bloom D, Kruper J, Rokem A, Yeatman JD. Development
  of the visual white matter pathways mediates development of
  electrophysiological responses in visual cortex. Hum Brain Mapp. 2021.
- Caffarra S, Kanopka K, Kruper J, Richie-Halford A, Roy E, Rokem A,
  Yeatman JD. Development of the alpha rhythm is linked to visual white
  matter pathways and visual detection performance. bioRxiv.
- Alexander LM, Escalera J, Ai L, et al. An open resource for
  transdiagnostic research in pediatric mental health and learning
  disorders. Sci Data. 2017.
- Richie-Halford A, Cieslak M, Ai L, et al. An analysis-ready and quality
  controlled resource for pediatric brain white-matter research. Scientific
  Data. 2022.
- Cieslak M, Cook PA, He X, et al. QSIPrep: an integrative platform for
  preprocessing and reconstructing diffusion MRI data. Nat Methods. 2021.
